In [68]:
from tkinter import *
from os.path import exists
import pandas as pd
import numpy as np
from odbm.odbm_main import ModelBuilder, ModelHandler
from odbm.utils import extractParams, fmt
from odbm import events
import matplotlib
matplotlib.use('TkAgg')
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure


To add:
- Sensitivity Analysis
    - user types in species and range of concentrations: A = 1:100, B = .1:1000

CURRENT ISSUE: 
setConstantParams doesn't actually set parameters to be constant, just changes there initial value


In [75]:
# Checks if model path given by user exists and builds model.
def loadModel(model_path,root):
    global myModel #create global variable so it is accessible in other functions
    global MH

    if exists(model_path) and model_path.endswith(('.xls','.xlsx')):
        model_species = pd.read_excel(model_path, sheet_name = 'Species & Base Mechanisms', engine = 'openpyxl').dropna('index','all')
        model_rxns = pd.read_excel(model_path, sheet_name = 'Reaction', engine = 'openpyxl').dropna('index','all')
        myModel = ModelBuilder(model_species, model_rxns)
        
        frame, root = newWindow(root)

        modelfile = saveModelTxt(myModel,model_path, frame)
    
        with open(modelfile,'r') as m:
            MH = ModelHandler(m.read())
        #print('Model successfully built and saved!')
        modelWindow(myModel, model_path, root)

    else:
        if exists(model_path):
            raise TypeError('Please pass in an excel compatible workbook with sheets named "Species & Base Mechanisms" and "Reaction"')
        else:
            raise FileExistsError('Error: No file exists with given name.')

In [80]:
# 2nd Window
def modelWindow(myModel, model_path, root):

    frame, root = newWindow(root)

        
    welcome = Label(frame, text = "Model successfully loaded and saved!")
    welcome.config(font = ('Arial', 12))
    welcome.pack()
    
    cancel_button = Button(frame, text="Cancel", fg="blue",command = lambda: cancelButton(root)) 
    #save_button = Button(frame, text = 'Save Model', fg = 'blue', command = lambda: saveModelTxt(myModel,model_path,frame))
    simulate_button = Button(frame, text = 'Simulate Model', fg = 'blue', command = lambda: runSimulation(myModel, model_path, root)) 
    edit_button = Button(frame, text = 'Edit Model', fg = 'blue', command = lambda: EditWindow(model_path,root)) 

    cancel_button.pack(side = LEFT)
    #save_button.pack(side = LEFT)
    simulate_button.pack(side = LEFT)
    edit_button.pack(side = LEFT)


In [84]:
def cancelButton(root):
    root.destroy()
    __init__()

def newWindow(root):
    root.destroy()
    root = Tk()
    frame = Frame(root)
    frame.pack()
    return frame, root


def saveModelTxt(myModel,model_path,frame):
    modelfile = model_path[:-4] + 'txt'
    myModel.saveModel(modelfile)
    #save_msg = Label(frame, text = 'Model successfully saved.')
    #save_msg.pack(side = BOTTOM)
    return modelfile


def runSimulation(myModel, model_path, root):

    # Close previous window and open new
    frame, root = newWindow(root)

    # Set up ModelHandler instance and selections for simulation
    #modelfile = saveModelTxt(myModel,model_path, frame)
    #with open(modelfile,'r') as m:
    #    MH = ModelHandler(m.read())
    selections = list(map(fmt, myModel.species['Label'])) + list(myModel.rxns['Label']) + ['time']


    #Add entries, buttons, labels
    start = Entry(frame, width = 10)
    end = Entry(frame, width = 10)
    points = Entry(frame, width = 10)
    titles = Label(frame, text = "Start: \t End: \t Points: \n")
    run_button = Button(frame, text = 'Run Simulation',  fg = 'blue', 
    command = lambda : [MH.setSimParams(int(start.get()),int(end.get()),int(points.get()), selections=selections), 
                        simHelper(MH, model_path, root)])
    back = Button(frame, text = 'Back', fg = 'Blue', command = lambda: modelWindow(myModel, model_path, root))
    
    titles.pack(side = LEFT)
    start.pack(side = LEFT)
    end.pack(side = LEFT)
    points.pack(side = LEFT)
    run_button.pack(side = RIGHT)
    back.pack(side = RIGHT)

def simHelper(MH, model_path, root):
    # Close previous window and open new
    frame, root = newWindow(root)


    results = MH.rr.simulate(MH.SimParams['start'],MH.SimParams['end'],MH.SimParams['points'],MH.SimParams['selections'])
    feedback = Label(frame, text = "Simulation successfully run")
    feedback.config(font = ('Arial', 10))
    feedback.pack()

    #Add entries, buttons, labels
    back = Button(frame, text = 'Back',  fg = 'blue', command = lambda: runSimulation(myModel,model_path,root))
    save = Button(frame,text = "Save Simulation Results", fg = 'blue', command = lambda: saveSim(results, model_path,frame)) 
    visualize = Button(frame, text = 'Visualize Simulation Results', fg = 'blue', command = lambda: visualizeSim(results,root)) 

    back.pack()
    save.pack()
    visualize.pack()

def saveSim(results, model_path,frame):
    sim_df = pd.DataFrame(results, columns = results.colnames).round(2)
    results_path = model_path[:-4] + '_RESULTS.xlsx'
    sim_df.to_excel(results_path, sheet_name = 'Results')
    # bug here right now but can't figure it out yet

    feedback = Label(frame, text = "Results successfully saved to excel file")
    feedback.config(font = ('Arial', 10))
    feedback.pack()
    
def visualizeSim(results, root):
#use vis.py, let user input reactions and/or species
    frame, root = newWindow(root)

    time_prompt = Label(frame, text = 'Time to plot over (start:end)')
    time = Entry()
    species_prompt = Label(frame, text = 'Species to plot, separated by commas')
    species = Entry()
    vis = Button(frame, text = 'Plot', fg = 'blue', command = lambda : visHelper(results, time.get(),species.get()))

    time_prompt.pack()
    time.pack()
    species_prompt.pack()
    species.pack()
    vis.pack()

def visHelper(results, time,species):
    start, end = time.split(':')
    t_list = np.linspace(start = float(start),stop = float(end),num = 100)
    species_list = species.split(',')

    window= Tk()
    fig = Figure(figsize=(6,6))
    
    for sp in species_list:

        a = fig.add_subplot(111)
        a.plot(t_list,results[sp])
        
    a.legend(labels = species_list)
    a.set_xlabel('Time')
    a.set_ylabel('Concentration')
    canvas = FigureCanvasTkAgg(fig, master=window)
    canvas.get_tk_widget().pack()
    canvas.draw()

    window.mainloop()


def addEvent(model_path,root):
    frame, root = newWindow(root)


    event_label = Label(frame, text = 'Enter event(s):')
    event = Entry()
    trigger_label = Label(frame, text = 'Enter event trigger:')
    trigger = Entry()
    add = Button(frame, text = 'Add Event', command = lambda : addEventHelper(event.get(),trigger.get(), frame))
    back = Button(frame, text = 'Back', command = lambda : EditWindow(model_path,root))

    event_label.pack(side = LEFT)
    event.pack(side = LEFT)
    trigger_label.pack(side = RIGHT)
    trigger.pack(side = RIGHT)
    add.pack()
    back.pack()

def addEventHelper(event,trigger,frame):
    events.addEvent(MH.rr, label = 'event', event = event, trigger = trigger)
    events.forceRegenerate(MH.rr)
    feedback = Label(frame, text = 'Event successfully added.')
    feedback.pack()






In [99]:
def EditWindow(model_path,root):
    
    frame, root = newWindow(root)

    constant = Button(frame, text = 'Add Constant Species', fg = 'blue', command = lambda: setConstantHelper(model_path, root))
    event = Button(frame,text = 'Add Event', fg = 'blue', command = lambda: addEvent(model_path, root))
    sensitivity = Button(frame, text = 'Sensitivity Analysis', fg = 'blue')
    back = Button(frame, text = 'Back',  fg = 'blue', command = lambda: modelWindow(myModel,model_path,root))


    constant.pack()
    event.pack()
    sensitivity.pack()
    back.pack()


def setConstant(frame,entry):
    param_dict = {}
    params = entry.split(',')
    for p in params:
        sp,val = p.split(':')
        param_dict[sp] = val
    print(param_dict)
    MH.setConstantParams(param_dict)
    for p,v in MH.ConstantParams.items():
        MH.rr[p] = float(v)

    
    feedback = Label(frame, text = "Values successfully set")
    feedback.config(font = ('Arial', 10))
    feedback.pack()
    


def setConstantHelper(model_path,root):

    frame, root = newWindow(root)

    prompt = Label(frame, text = "Insert list of species and concentrations in format species:concentration separated by commas")
    prompt.config(font = ('Arial', 12))
    entry = Entry(frame, width = 40)
    set = Button(frame, text = 'Set', fg = 'blue', command = lambda : setConstant(frame,entry.get()))
    back = Button(frame, text = 'Back',  fg = 'blue', command = lambda: EditWindow(model_path,root))

    prompt.pack()
    entry.pack()
    set.pack()
    back.pack()
    
    

In [110]:
# Main Window
def __init__():
    root = Tk()
    frame = Frame(root)
    frame.pack()

    welcome = Label(frame, text = "Welcome to ODBM \n Please insert file path for model to load:")
    welcome.config(font = ('Arial', 12))
    entry = Entry(frame, width = 40)
    loadbutton = Button(frame, text="Load Model", fg="blue", command = lambda:loadModel(entry.get(),root))

    welcome.pack()
    loadbutton.pack(side = BOTTOM)
    entry.pack()
    root.mainloop()

__init__()

{'A': '10'}


C:\Users\rycar\anaconda3\lib\site-packages\ipykernel_launcher.py:105: MatplotlibDeprecationWarning:

Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.



In [51]:
C:\Users\rycar\Documents\GitHub\ODBM\Examples\model_definition.xlsx

SyntaxError: unexpected character after line continuation character (<ipython-input-51-cf614fb1cf2e>, line 1)

In [2]:
model_path = 'C:/Users/rycar/Documents/GitHub/ODBM/Examples/model_definition.xlsx'
model_species = pd.read_excel(model_path, sheet_name = 'Species & Base Mechanisms', engine = 'openpyxl').dropna('index','all')
model_rxns = pd.read_excel(model_path, sheet_name = 'Reaction', engine = 'openpyxl').dropna('index','all')
myModel = ModelBuilder(model_species, model_rxns)
modelfile = 'C:/Users/rycar/Documents/GitHub/ODBM/Examples/model_definition.txt'
with open(modelfile,'r') as m:
        MH = ModelHandler(m.read())
selections = list(map(fmt, myModel.species['Label'])) + list(myModel.rxns['Label']) + ['time']
MH.setSimParams(0,100,100, selections = selections)
results = MH.rr.simulate(MH.SimParams['start'],MH.SimParams['end'],MH.SimParams['points'],MH.SimParams['selections'])



In [3]:
from odbm import events

In [11]:
#events.removeEvent(MH.rr, label = 'test')
events.addEvent(MH.rr, label = 'test', event = 'A = 10', trigger = 'time > 10')
#events.forceRegenerate(MH.rr)

In [8]:
MH.setSimParams(0,100,100, selections = selections)
results = MH.rr.simulate(MH.SimParams['start'],MH.SimParams['end'],MH.SimParams['points'],MH.SimParams['selections'])

In [9]:
results

          A,          B,       C,       R1,       R2,         R3,          R4,    time
 [[ 7.02439,  0.0033135,  2.4102, 0.232753,  0.24102,   0.006627,  0.00165675,       0],
  [ 7.03605, 0.00329677, 2.40188, 0.231962, 0.240188, 0.00659353,  0.00164838,  1.0101],
  [ 7.04766, 0.00328016, 2.39359, 0.231175, 0.239359, 0.00656032,  0.00164008,  2.0202],
  [ 7.05921,  0.0032637, 2.38534, 0.230391, 0.238534, 0.00652739,  0.00163185,  3.0303],
  [ 7.07071, 0.00324736, 2.37714, 0.229611, 0.237714, 0.00649472,  0.00162368,  4.0404],
  [ 7.08214, 0.00323116, 2.36897, 0.228835, 0.236897, 0.00646232,  0.00161558, 5.05051],
  [ 7.09352, 0.00321509, 2.36085, 0.228063, 0.236085, 0.00643018,  0.00160754, 6.06061],
  [ 7.10485, 0.00319915, 2.35277, 0.227295, 0.235277, 0.00639829,  0.00159957, 7.07071],
  [ 7.11611, 0.00318333, 2.34472,  0.22653, 0.234472, 0.00636666,  0.00159167, 8.08081],
  [ 7.12733, 0.00316764, 2.33672, 0.225768, 0.233672, 0.00633529,  0.00158382, 9.09091],
  [ 9.99995, 0.00227312

In [109]:
MH.rr.setBoundarySpeciesConcentrations(len = 100, indx = 0, values = 10)

AttributeError: 'RoadRunner' object has no attribute 'setBoundarySpeciesConcentrations'

NameError: name 'points' is not defined